## Dueling Deep Reinforcement Learning with Praoritized replay

In [1]:
!apt-get update

Get:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [3724 B]
Get:2 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6384 B] 
Get:3 http://packages.cloud.google.com/apt cloud-sdk InRelease [6349 B]        
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Ign:8 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:10 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:11 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://packages.c

In [2]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux
apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

# Lua binding dependencies
apt-get install liblua5.1-dev

Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.4ubuntu1).
libjpeg-dev is already the newest version (8c-2ubuntu8).
unzip is already the newest version (6.0-21ubuntu1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
git is already the newest version (1:2.17.1-1ubuntu0.7).
tar is already the newest version (1.29b-2ubuntu0.1).
The following additional packages will be installed:
  autoconf automake autopoint bsdmainutils bzip2-doc dbus debhelper
  dh-autoreconf dh-strip-nondeterminism freepats gettext gettext-base
  gir1.2-atk-1.0 gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-gtk-2.0
  gir1.2-ibus-1.0 gir1.2-pango-1.0 groff-base intltool-debian
  libarchive-cpio-perl libarchive-zip-perl libasound2-dev libatk1.0-dev
  libaudio2 libcairo-script-interpreter2 libcairo2-dev libcapnp-0.6.1
  libdbus-1-3 libdbus-1-dev libdrm-dev 

debconf: delaying package configuration, since apt-utils is not installed


In [3]:
!pip install vizdoom

     |████████████████████████████████| 4.9 MB 2.9 MB/s eta 0:00:01
  Created wheel for vizdoom: filename=vizdoom-1.1.7-py3-none-any.whl size=14312184 sha256=35725e46739b18e54cd54672ab1e13154e7d1035ec068c2376f90c6480b8f4c6
  Stored in directory: /root/.cache/pip/wheels/e3/78/dd/a11a3b2e29e59f1756d8d1cf581026ac30fabbb5a037f94a47
Successfully built vizdoom


In [5]:
!git clone https://github.com/simoninithomas/Deep_reinforcement_learning_Course.git

fatal: destination path 'Deep_reinforcement_learning_Course' already exists and is not an empty directory.


In [6]:
from __future__ import division
from __future__ import print_function
import itertools as it
from random import sample, randint, random
from time import time, sleep
import numpy as np
import skimage.color, skimage.transform
import tensorflow as tf
from tqdm import trange
import vizdoom as vzd
from argparse import ArgumentParser

In [7]:
!cat "Deep_reinforcement_learning_Course/Dueling Double DQN with PER and fixed-q targets/deadly_corridor.cfg"

# Lines starting with # are treated as comments (or with whitespaces+#).
# It doesn't matter if you use capital letters or not.
# It doesn't matter if you use underscore or camel notation for keys, e.g. episode_timeout is the same as episodeTimeout.

doom_scenario_path = deadly_corridor.wad

# Skill 5 is reccomanded for the scenario to be a challenge.
doom_skill = 5

# Rewards
death_penalty = 100
#living_reward = 0

# Rendering options
screen_resolution = RES_160X120
screen_format = GRAY8
render_hud = true
render_crosshair = false
render_weapon = true
render_decals = false
render_particles = false
window_visible = true

episode_timeout = 2100

# Available buttons
available_buttons = 
	{ 
		MOVE_LEFT 
		MOVE_RIGHT 
		ATTACK 
		MOVE_FORWARD
		MOVE_BACKWARD
		TURN_LEFT
		TURN_RIGHT
	}

# Game variables that will be in the state
available_game_variables = { HEALTH }

mode = PLAYER




In [8]:
#DEFAULT_MODEL_SAVEFILE = "drive/My Drive/Colab Notebooks/tmp/model"
DEFAULT_CONFIG = "Deep_reinforcement_learning_Course/Dueling Double DQN with PER and fixed-q targets/deadly_corridor.cfg"
DEFAULT_SCN = "Deep_reinforcement_learning_Course/Dueling Double DQN with PER and fixed-q targets/deadly_corridor.wad"

In [9]:
# Creates and initializes ViZDoom environment.
def initialize_vizdoom(config_file_path , scenario_file_path):
    print("Initializing doom...")
    game = vzd.DoomGame()
    game.load_config(config_file_path)
    game.set_doom_scenario_path(scenario_file_path)
    game.set_window_visible(False)
    game.set_mode(vzd.Mode.PLAYER)
    game.set_screen_format(vzd.ScreenFormat.GRAY8)
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.init()
    print("Doom initialized.")
    return game

In [10]:
doom=initialize_vizdoom(DEFAULT_CONFIG , DEFAULT_SCN)

Initializing doom...
Doom initialized.


In [11]:
import random                # Handling random number generation
import time                  # Handling time calculation
from skimage import transform# Help us to preprocess the frames

from collections import deque# Ordered collection with ends
from collections import namedtuple
import torch
import torch.optim as optim
import numpy as np
stacked_size = 4

def preprocess(frame):
    """
    screen frames are in grayscale format defalut
    1.normalize the images
    2. apply some transformations
    """
    frame = frame / 255.0
    frame = transform.resize(frame ,[96,96])
    
    return frame
    
    
def stack_frames(stacked_frames , frame , new_episode=False):
    """
    stack multiple frames with each other to idenitify the temporal movemnts of the objects
    1. preprocess the new frame
    """
    processed_frame = preprocess(frame)
    if(new_episode):
        stacked_frames = deque([ np.zeros([96,96] ,dtype=int) for i_dx in range(stacked_size) ] , maxlen=stacked_size)
        #for initial step after new episode add same frame to all the stacked frames
        stacked_frames.append(processed_frame)
        stacked_frames.append(processed_frame)
        stacked_frames.append(processed_frame)
        stacked_frames.append(processed_frame)
        
        stack_states = np.stack(stacked_frames , axis=0)
        
        return stack_states , stacked_frames
    
    else:
        stacked_frames.append(processed_frame)
        
        stack_states = np.stack(stacked_frames , axis=0)
        
        return stack_states , stacked_frames

## Define the SumTree Structure for the Replay buffer

In [12]:
import numpy


# SumTree
# a binary tree data structure where the parent’s value is the sum of its children
class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = numpy.zeros(2 * capacity - 1)
        self.data = numpy.zeros(capacity, dtype=object)
        self.n_entries = 0

    # update to the root node
    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    # find sample on leaf node
    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    # store priority and sample
    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

        if self.n_entries < self.capacity:
            self.n_entries += 1

    # update priority
    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    # get priority and sample
    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])

## Define the prioritized raply memory

In [13]:
import random
import numpy as np

class ReplayBuffer :  # stored as ( s, a, r, s_ ) in SumTree
    e = 0.01
    a = 0.6
    beta = 0.4
    beta_increment_per_sampling = 0.001

    def __init__(self, capacity):
        self.tree = SumTree(capacity)
        self.capacity = capacity
        self.experience = namedtuple('Experience' , field_names=['state','action','reward','next_state','done'])

    def _get_priority(self, error):
        return (np.abs(error) + self.e) ** self.a

    def add(self, error, state , action , reward , next_state , done):
        p = self._get_priority(error)
        experience = self.experience(state , action , reward , next_state , done)
        self.tree.add(p, experience)

    def sample(self, n):
        try:
            batch = []
            idxs = []
            segment = self.tree.total() / n
            priorities = []

            self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])

            for i in range(n):
                a = segment * i
                b = segment * (i + 1)

                s = random.uniform(a, b)
                (idx, p, data) = self.tree.get(s)
                priorities.append(p)
                batch.append(data)
                idxs.append(idx)

            states = torch.from_numpy( np.stack([ e.state.reshape(4,96,96) for e in batch if e is not None ],axis=0) ).float().to(device)
            action = torch.from_numpy( np.stack([ (e.action).reshape(1,) for e in batch if e is not None] , axis=0) ).long().to(device)
            reward = torch.from_numpy( np.stack([ np.array((e.reward)).reshape(1,) for e in batch if e is not None] , axis=0) ).float().to(device)
            next_state = torch.from_numpy( np.stack([ e.next_state.reshape(4,96,96) for e in batch if e is not None ] , axis=0) ).float().to(device)
            done = torch.from_numpy( np.stack([ np.array((e.done)).reshape(1,) for e in batch if e is not None], axis=0).astype(np.uint8) ).float().to(device)

            sampling_probabilities = priorities / self.tree.total()
            is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
            is_weight /= is_weight.max()

            return (states , action , reward , next_state , done) , idxs, is_weight
        except :
            print(batch)

    def update(self, idx, error):
        p = self._get_priority(error)
        self.tree.update(idx, p)
        
    def __len__(self):
        return self.tree.n_entries

In [14]:
BATCH_SIZE = 16
UPDATE_EVERY = 4
SEED= 1244
BUFFER_SIZE=int(2e4)
GAMMA= 0.99
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
class Agent :
    def __init__(self,state_size , action_size , seed):
        
        self.state_size  = state_size 
        self.action_size = action_size
        self.seed  = random.seed(seed)
        self.train_loss = deque(maxlen=500)
        """
        step --> add the data to experience tuple and if in the update instance train the model
        act --> output the action for states
        learn --> train the model using experince tuple
        soft_update --> update the target network 
        """
        """
        1 --> define local and target  Q networks
        2 --> initialize the optimizer
        3 --> initialize the raply buffer
        
        """
        self.q_local = DeepQNet(self.state_size , self.action_size , seed).to(device)
        self.q_target = DeepQNet(self.state_size , self.action_size , seed).to(device)
        self.optimizer = optim.Adam(self.q_local.parameters() , lr=LR)
        
        self.replay_buffer = ReplayBuffer(BUFFER_SIZE)
        self.t_step = 0    # for determine the update instances
        
    def step(self,state , action , reward , next_state , done , fill_buffer=False):
        
        # need to calculate the error for the each sample
        self.q_local.eval()
        self.q_target.eval()
        with torch.no_grad():
            target = self.q_local(Variable(torch.FloatTensor(state).unsqueeze(0).to(device) )).data
            old_val = target[0][action]
            target_val = self.q_target(Variable(torch.FloatTensor(next_state).unsqueeze(0).to(device) )).data
        if done:
            target[0][action] = reward
        else:
            target[0][action] = reward + GAMMA * torch.max(target_val)

        error = abs(old_val - target[0][action])
        error = error.detach().to('cpu').numpy()
        
        self.q_local.train()
        self.q_target.train()
        
        self.replay_buffer.add(error,state , action , reward , next_state , done)
        self.t_step = (self.t_step + 1)%UPDATE_EVERY
        if( not fill_buffer):
            if(self.t_step == 0):
                if(len(self.replay_buffer) > BATCH_SIZE):
                    #get the training batch from the tree structuer raply buffer
                    train_sample ,  idxs, is_weight = self.replay_buffer.sample(BATCH_SIZE)

                    self.learn(train_sample , idxs , is_weight , GAMMA)
                
    def learn(self , train_sample ,idxs , is_weights , GAMMA) :
        state , action , reward , next_state , done = train_sample
        """
        1 --> infernce the local model with state and take the Q values for actions associated
        2 --> indernce the target model and get the maximum Q value
        3 --> obtain the q target with ( reward + GAMMA * max(Q_target_model(next_state))*(1-dones))
        4 --> MSELoss(Q_local , Q_target)
        5 --> optimize the model
        """
        Q_expected = self.q_local(state).gather(1,action)
        Q_target = self.q_target(next_state).detach().max(1)[0].unsqueeze(1)
        Q_target = reward + GAMMA*(Q_target*(1-done))
        #calculate errors for the tree update
        errors = torch.abs(Q_expected - Q_target).data
        errors = errors.detach().to('cpu').numpy()
        # update priority
        for i in range(BATCH_SIZE):
            idx = idxs[i]
            self.replay_buffer.update(idx, errors[i])
            
        
        
        q_loss = (torch.FloatTensor(is_weights).to(device) * F.mse_loss(Q_expected, Q_target)).mean()
        
        self.train_loss.append(q_loss.to('cpu').detach().item())
        # reset the optimizer
        self.optimizer.zero_grad()
        q_loss.backward()
        self.optimizer.step()
        
        #update the weights of target model weights
        self.soft_update(self.q_target , self.q_local ,TAU)
        
    def act(self , state , eps=0.):
        """
        expand the dim 0 of the state tensor
        base on eps either select max action or a random action
        """
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.q_local.eval()
        with torch.no_grad():
            action_tensor = self.q_local(state_tensor)
        self.q_local.train()
        
        # if the random.random > eps --> select a random action else select greedy action
        if(random.random() > eps ):
            return np.argmax(action_tensor.to('cpu').detach().numpy())
        
        else :
            return random.choice(np.arange(self.action_size))
        
    def soft_update(self , target_model , local_model , tau) :
        
        for target_param , local_param in zip(target_model.parameters() , local_model.parameters()):
            
            target_param.data.copy_(tau*local_param.data + (1.-tau)*target_param.data)
        

In [15]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [16]:
def conv_block(in_channels , out_channles , kernel_size =3 , stride=1 , padding=1 , batch_norm=True , maxpool=False ):
    layers =[]
    conv_layer = nn.Conv2d(in_channels=in_channels , out_channels=out_channles , kernel_size=kernel_size ,
                          stride=stride , padding=padding , padding_mode='reflect', bias=False)
    layers.append(conv_layer)
    
    if(batch_norm):
        bn = nn.BatchNorm2d(out_channles)
        layers.append(bn)
    layers.append(nn.ReLU())
    
    if(maxpool):
        max_layer = nn.MaxPool2d(kernel_size=4, stride=2 , padding=1)
        layers.append(max_layer)
        
    return nn.Sequential(*layers)

## Dueling Deep Q Network
![](https://miro.medium.com/max/1600/1*GKZ-cS0mCdXMOO_bfBlN0Q.png)

In [17]:
class DeepQNet(nn.Module):
    def __init__(self,stack_size , action_size , seed):
        super(DeepQNet , self).__init__()
        """
        define a simple model with some conv layers and later with fully connected layers
        """
        self.in_size = stack_size
        self.out_size = action_size
        self.seed = torch.manual_seed(seed)
        # 224 * 224 * 4 --> 224 * 224 * 32
        self.conv_block1 = conv_block(self.in_size , 32 )
        # 224 * 224 * 32 --> 112 * 112 * 64
        self.conv_block2 = conv_block(32 , 64 , maxpool=True)
        #112 * 112 * 64 --> 112 * 112 * 128
        self.conv_block3 = conv_block(64 , 128 )
        # 112 * 112 * 128 --> 56 * 56 * 256
        self.conv_block4 = conv_block(128,256,maxpool=True)
        # 56 * 56 * 256 --> 56 * 56 * 512
        self.conv_block5 = conv_block(256 , 512)
        # 56 * 56 * 512 --> 28 * 28 * 1024 
        self.conv_block6 = conv_block(512 , 1024 , maxpool=True)
        self.flatten_size = 12*12*1024
        
        #state value stream
        self.fc_value = nn.Linear(self.flatten_size , 512)
        self.fc_value_bn = nn.BatchNorm1d(512)
        self.fc_value_out = nn.Linear(512 , 1 )
        
        #advantage stream
        self.fc_advantage = nn.Linear(self.flatten_size , 512)
        self.fc_advantage_bn = nn.BatchNorm1d(512)
        self.fc_advantage_out = nn.Linear(512 , self.out_size)
        
        
        
    def forward(self, x):

        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = self.conv_block6(x)
        x = x.view(-1,self.flatten_size)
        #state value stream
        state_value = F.dropout(F.relu(self.fc_value_bn(self.fc_value(x))) ,p=0.4 )
        state_value = self.fc_value_out(state_value) 
        
        #state advantage stream
        advantage_value = F.dropout(F.relu(self.fc_advantage_bn(self.fc_advantage(x))) , p=0.4 )
        advantage_value = self.fc_advantage_out(advantage_value)
        
        output = state_value + torch.sub(advantage_value , torch.mean(advantage_value , dim=1 , keepdim=True))
        
        return output

In [18]:
agent = Agent(state_size=4, action_size=7, seed=1243)

In [ ]:
agent.q_local.load_state_dict(torch.load('../input/doom-rl/local_model.pth'))
agent.q_target.load_state_dict(torch.load('../input/doom-rl/target_model.pth'))

## Fill the memory buffer with rndom samples

In [19]:
game = initialize_vizdoom(DEFAULT_CONFIG , DEFAULT_SCN)
max_lenght=int(2e4)
step = 0
MOVE_LEFT  =[1,0,0,0,0,0,0]
MOVE_RIGHT =[0,1,0,0,0,0,0]
ATTACK     =[0,0,1,0,0,0,0]
MOVE_FORWARD=[0,0,0,1,0,0,0]
MOVE_BACKWARD=[0,0,0,0,1,0,0]
TURN_LEFT = [0,0,0,0,0,1,0]
TURN_RIGHT= [0,0,0,0,0,0,1]
actions = [MOVE_LEFT , MOVE_RIGHT , ATTACK , MOVE_FORWARD , MOVE_BACKWARD , TURN_LEFT , TURN_RIGHT]
possible_actions = np.arange(7)        
# Make a new episode and observe the first state
game.new_episode()
state = game.get_state().screen_buffer
            
# Remember that stack frame function also call our preprocess function.
state, stacked_frames = stack_frames(None, state, True)

while step < max_lenght:
    
    step += 1 
    # Predict the action to take and take it
    action  = random.choice(possible_actions)

    # Do the action
    reward = game.make_action(actions[action])

    # Look if the episode is finished
    done = game.is_episode_finished()
                

    # If the game is finished
    if done:
        # the episode ends so no next state
        next_state = np.zeros((96,96), dtype=np.int)
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        
        # Set step = max_steps to end the episode
        step +=1

        agent.step(state, action, reward, next_state, done  , fill_buffer=True)
        
        game.new_episode()
        state = game.get_state().screen_buffer
        # Remember that stack frame function also call our preprocess function.
        state, stacked_frames = stack_frames(None, state, True)

    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
                    
        # Stack the frame of the next_state
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                
        # Add experience to memory
        agent.step(state, action, reward, next_state, done , fill_buffer=True)
                    
        # st+1 is now our current state
        state = next_state

Initializing doom...
Doom initialized.


In [20]:
# Init the game
MOVE_LEFT  =[1,0,0,0,0,0,0]
MOVE_RIGHT =[0,1,0,0,0,0,0]
ATTACK     =[0,0,1,0,0,0,0]
MOVE_FORWARD=[0,0,0,1,0,0,0]
MOVE_BACKWARD=[0,0,0,0,1,0,0]
TURN_LEFT = [0,0,0,0,0,1,0]
TURN_RIGHT= [0,0,0,0,0,0,1]
actions = [MOVE_LEFT , MOVE_RIGHT , ATTACK , MOVE_FORWARD , MOVE_BACKWARD , TURN_LEFT , TURN_RIGHT]
total_episodes=3500
game = initialize_vizdoom(DEFAULT_CONFIG , DEFAULT_SCN)
explore_probability = 1.0
explore_probability_end=0.01
eps_decay = 0.99995
max_steps = 5000
for episode in range(total_episodes):
    # Set step to 0
    step = 0
            
    # Initialize the rewards of the episode
    episode_rewards = []
            
    # Make a new episode and observe the first state
    game.new_episode()
    state = game.get_state().screen_buffer
            
    # Remember that stack frame function also call our preprocess function.
    state, stacked_frames = stack_frames(None, state, True)

    while step < max_steps:
        step += 1 
        # Predict the action to take and take it
        action  = agent.act(state , eps=explore_probability)

        # Do the action
        reward = game.make_action(actions[action])

        # Look if the episode is finished
        done = game.is_episode_finished()
                
        # Add the reward to total reward
        episode_rewards.append(reward)

        # If the game is finished
        if done:
            # the episode ends so no next state
            next_state = np.zeros((96,96), dtype=np.int)
            next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

            # Set step = max_steps to end the episode
            step = max_steps

            # Get the total reward of the episode
            total_reward = np.sum(episode_rewards)
            if(episode%10==0):
                print('Episode: {}'.format(episode),
                        'Total reward: {}'.format(total_reward),
                        'Training loss: {:.4f}'.format(np.mean(agent.train_loss)),
                        'Explore P: {:.4f}'.format(explore_probability))

            agent.step(state, action, reward, next_state, done)

        else:
            # Get the next state
            next_state = game.get_state().screen_buffer
                    
            # Stack the frame of the next_state
            next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    
            # Add experience to memory
            agent.step(state, action, reward, next_state, done)
                    
            # st+1 is now our current state
            state = next_state
    
    explore_probability = max(explore_probability_end, explore_probability*eps_decay)

Initializing doom...
Doom initialized.
Episode: 0 Total reward: -115.99937438964844 Training loss: 311.6106 Explore P: 1.0000
Episode: 10 Total reward: -115.99937438964844 Training loss: 203.1262 Explore P: 0.9995
Episode: 20 Total reward: -115.99856567382812 Training loss: 120.6275 Explore P: 0.9990
Episode: 30 Total reward: -114.97325134277344 Training loss: 54.2187 Explore P: 0.9985
Episode: 40 Total reward: -115.98001098632812 Training loss: 36.2314 Explore P: 0.9980
Episode: 50 Total reward: -115.99006652832031 Training loss: 28.5947 Explore P: 0.9975
Episode: 60 Total reward: -107.61994934082031 Training loss: 21.4248 Explore P: 0.9970
Episode: 70 Total reward: -72.72604370117188 Training loss: 18.8234 Explore P: 0.9965
Episode: 80 Total reward: -115.12837219238281 Training loss: 14.8702 Explore P: 0.9960
Episode: 90 Total reward: -115.78532409667969 Training loss: 11.0211 Explore P: 0.9955
Episode: 100 Total reward: -115.22006225585938 Training loss: 10.9928 Explore P: 0.9950
Ep

Episode: 940 Total reward: -106.75836181640625 Training loss: 16.0556 Explore P: 0.9541
Episode: 950 Total reward: -114.36772155761719 Training loss: 14.6637 Explore P: 0.9536
Episode: 960 Total reward: -90.78187561035156 Training loss: 13.9716 Explore P: 0.9531
Episode: 970 Total reward: -99.14569091796875 Training loss: 13.9682 Explore P: 0.9527
Episode: 980 Total reward: -101.15765380859375 Training loss: 15.4715 Explore P: 0.9522
Episode: 990 Total reward: -103.78269958496094 Training loss: 15.3418 Explore P: 0.9517
Episode: 1000 Total reward: -108.02047729492188 Training loss: 14.9543 Explore P: 0.9512
Episode: 1010 Total reward: -114.48788452148438 Training loss: 15.2462 Explore P: 0.9508
Episode: 1020 Total reward: -112.69207763671875 Training loss: 18.8136 Explore P: 0.9503
Episode: 1030 Total reward: -113.12254333496094 Training loss: 20.1224 Explore P: 0.9498
Episode: 1040 Total reward: -94.54820251464844 Training loss: 16.6420 Explore P: 0.9493
Episode: 1050 Total reward: -9

KeyboardInterrupt: 

In [21]:
torch.save(agent.q_local.state_dict(),'local_model.pth')
torch.save(agent.q_target.state_dict(),'target_model.pth')

In [23]:
!rm -rf ./Deep_reinforcement_learning_Course

In [ ]:
img_frames = []
max_steps=500
game = initialize_vizdoom(DEFAULT_CONFIG , DEFAULT_SCN)
# Make a new episode and observe the first state
game.new_episode()

state = game.get_state().screen_buffer
img_frames.append(state)          
# Remember that stack frame function also call our preprocess function.
state, stacked_frames = stack_frames(None, state, True)
agent.q_local.eval()
step=0
while step < max_steps:
    step += 1 
    # Predict the action to take and take it
    state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
    with torch.no_grad():
        action_tensor = agent.q_local(state_tensor)
    action = np.argmax(action_tensor.to('cpu').detach().numpy()) 
    # Do the action
    reward = game.make_action(actions[action])

    # Look if the episode is finished
    done = game.is_episode_finished()
                
    # Add the reward to total reward
    episode_rewards.append(reward)
    # If the game is finished
    if done:
        # the episode ends so no next state
        next_state = np.zeros((96,96), dtype=np.int)
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

        # Set step = max_steps to end the episode
        step = max_steps

        # Get the total reward of the episode
        total_reward = np.sum(episode_rewards)
        print('Total reward: {}'.format(total_reward))

    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
        img_frames.append(next_state)        
        # Stack the frame of the next_state
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    
        # st+1 is now our current state
        state = next_state

In [ ]:
data_path='data'
import PIL
for idx , image  in enumerate(img_frames):
 PIL.Image.fromarray(image).save('data/{}.png'.format(str(idx)))

In [ ]:
import os
os.system('ffmpeg -r 10 -i data/%1d.png -vcodec libx264 -b 10M -y FlowVideo.mp4  ')

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('FlowVideo.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)